In [1]:
pip show pyspark

Name: pyspark
Version: 3.5.4
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages
Requires: py4j
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark import SparkContext

# Initialize SparkContext
sc = SparkContext(master="local", appName="CreateRDD")

24/12/26 19:53:46 WARN Utils: Your hostname, Chirags-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.11 instead (on interface en0)
24/12/26 19:53:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/26 19:53:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.master("local").appName("CSVExample").getOrCreate()

# Load CSV into a DataFrame
file_path = "2015-summary.csv"  # Replace with the actual path
flight_data_df = spark.read.option("header", "true").csv(file_path)

# Display the DataFrame schema
flight_data_df.printSchema()

# Show the first few rows
flight_data_df.show(5)
type(flight_data_df)


root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



pyspark.sql.dataframe.DataFrame

In [4]:
# Read the CSV file using DataSource API
flight_data_df = spark.read.format("csv").option("header", "true").load(file_path)

# Display the DataFrame schema
flight_data_df.printSchema()

# Show the first few rows
flight_data_df.show(5)
print(type(flight_data_df))
flight_data_rdd = flight_data_df.rdd
print(type(flight_data_rdd))

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.rdd.RDD'>


In [5]:
# Read the file using SparkContext
file_rdd = sc.textFile(file_path)

# Display the first few lines
print(file_rdd.take(5))
print(type(file_rdd))

['DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count', 'United States,Romania,15', 'United States,Croatia,1', 'United States,Ireland,344', 'Egypt,United States,15']
<class 'pyspark.rdd.RDD'>


In [20]:
import pandas as pd

spth = '2015-summary.csv'
pd_flightData2015 = pd.read_csv(spth,header = 0)
print(type(pd_flightData2015))
print(pd_flightData2015.head(5))
pd_flightData2015 = spark.createDataFrame(pd_flightData2015).rdd
print(type(pd_flightData2015))
print(pd_flightData2015.take(5))

<class 'pandas.core.frame.DataFrame'>
  DEST_COUNTRY_NAME ORIGIN_COUNTRY_NAME  count
0     United States             Romania     15
1     United States             Croatia      1
2     United States             Ireland    344
3             Egypt       United States     15
4     United States               India     62
<class 'pyspark.rdd.RDD'>
[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15), Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1), Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344), Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15), Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]


In [23]:
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
myCollection
words = spark.sparkContext.parallelize(myCollection, 2)
words.take(10)


['Spark',
 'The',
 'Definitive',
 'Guide',
 ':',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

In [25]:
##### from a collection of text
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
words = spark.sparkContext.parallelize(myCollection, 2)
words.setName("myWords")
print(words.name())
print(type(words))
print(words.take(5))
words.getNumPartitions()



myWords
<class 'pyspark.rdd.RDD'>
['Spark', 'The', 'Definitive', 'Guide', ':']


2

In [9]:
#from range of numbers
myRange = spark.range(1000).toDF("number").rdd.map(lambda row: row[0])
print(myRange.take(5))
print(type(myRange))
print(myRange.getNumPartitions())

[0, 1, 2, 3, 4]
<class 'pyspark.rdd.PipelinedRDD'>
1


# Filtering

In [10]:
def parseLine(line):
    fields = line.split(',')#list
    date = fields[0]
    p_open = fields[1]
    p_close = fields[5]
    return (date, p_open, p_close)

In [11]:
spth="RELIANCE.csv"
sdt=spark.sparkContext.textFile(spth)
sdt=sdt.map(parseLine)
sdt.take(2)

[('Date', 'Open', 'Close'), ('2022-01-01', '100', '10000')]

In [12]:
spth="RELIANCE.csv"
o_sdt = spark.read.format("CSV").option("header","true").option("inferSchema", "true") \
    .load(spth)
o_sdt=o_sdt.toDF("Date","Open","High","Low","Last","Close","Volume","Turnover").rdd \
    .map(lambda row: (row[0], row[1], row[5]))
print(o_sdt.count())
print(type(o_sdt))
print(o_sdt.take(2))

o_sdt=o_sdt.filter(lambda row: row[2] > row[1])
print(o_sdt.take(5))
print(type(o_sdt))
print(o_sdt.count())

5
<class 'pyspark.rdd.PipelinedRDD'>
[(datetime.date(2022, 1, 1), 100, 10000), (datetime.date(2022, 1, 2), 200, 20000)]
[(datetime.date(2022, 1, 1), 100, 10000), (datetime.date(2022, 1, 2), 200, 20000), (datetime.date(2022, 1, 3), 300, 30000), (datetime.date(2022, 1, 4), 400, 40000), (datetime.date(2022, 1, 5), 500, 50000)]
<class 'pyspark.rdd.PipelinedRDD'>
5


# Filter Function

In [13]:
def HighClose(row):
    ## Discuss
    if row[2] > row[1]:
        return(row)
    

In [14]:
o_sdt = spark.read.format("CSV").option("header","true").load(spth)
o_sdt=o_sdt.toDF("Date","Open","High","Low","Last","Close","Volume","Turnover").rdd.map(lambda row: (row[0], row[1], row[5]))
print(o_sdt.count())
o_sdt=o_sdt.filter(lambda row: HighClose(row))
print(o_sdt.take(5))
print(type(o_sdt))
print(o_sdt.count())

5
[('2022-01-01', '100', '10000'), ('2022-01-02', '200', '20000'), ('2022-01-03', '300', '30000'), ('2022-01-04', '400', '40000'), ('2022-01-05', '500', '50000')]
<class 'pyspark.rdd.PipelinedRDD'>
5


In [15]:
o_sdt = spark.read.format("CSV").option("header","true").option("inferSchema", "true").load(spth)
o_sdt=o_sdt.toDF("Date","Open","High","Low","Last","Close","Volume","Turnover").rdd.map(lambda row: (row[0], row[1], row[5]))
print(o_sdt.count())
o_sdt=o_sdt.filter(lambda row: HighClose(row))
print(o_sdt.take(5))
print(type(o_sdt))
print(o_sdt.count())


5
[(datetime.date(2022, 1, 1), 100, 10000), (datetime.date(2022, 1, 2), 200, 20000), (datetime.date(2022, 1, 3), 300, 30000), (datetime.date(2022, 1, 4), 400, 40000), (datetime.date(2022, 1, 5), 500, 50000)]
<class 'pyspark.rdd.PipelinedRDD'>
5


# Maps

In [16]:
def to_to_mill(row):
    return (row[0], row[1], row[2], round(row[3],0))

In [17]:
spth="RELIANCE.csv"
o_sdt = spark.read.format("CSV").option("header","true").option("inferSchema", "true").load(spth)
o_sdt=o_sdt.toDF("Date","Open","High","Low","Last","Close","Volume","Turnover").rdd.map(lambda row: (row[0], row[1], row[5], row[7]))
print(o_sdt.take(2))
o_sdt=o_sdt.map(to_to_mill)
print(type(o_sdt))

[(datetime.date(2022, 1, 1), 100, 10000, None), (datetime.date(2022, 1, 2), 200, 20000, None)]
<class 'pyspark.rdd.PipelinedRDD'>


# flatMap

In [18]:
def Func(lines):
    lines = lines.lower()
    lines = lines.split(" ")
    return lines

#sc.stop()
#conf = SparkConf().setMaster("local").setAppName("wordcount")
#sc = SparkContext.getOrCreate()

spth="/Users/aakash/Downloads/spark-code/data/sherlock_holmes.txt"
input_file = sc.textFile(spth)